2. ROC 곡선 그려보기, Confusion matrix 출력
3. PCA로 차원 줄여보고 다시 앞의 방법으로 훈련시켜 정확도 출력

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
# 난수 초기화
np.random.seed(42)

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

X, y = mnist['data'], mnist['target']

In [ ]:
from sklearn.decomposition import PCA

rnd_pca = PCA(n_components=154, svd_solver="randomized", random_state=42) 
X_reduced = rnd_pca.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = X_reduced[:50000], X_reduced[50000:], y[:50000], y[50000:]

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV

extra_trees_clf = ExtraTreesClassifier(random_state=42)

param_grid = [{'n_estimators' : [350, 400, 450],
               'max_depth' : [8, 10, 12]}]
#1차 : [10,20,30,40,50]  =>  {'n_estimators': 50}
#       [8, 10, 12]  => 12
#2차 : [50, 100, 200, 300]  =>  {'n_estimators': 350}
#3차 : [350, 400, 450]
grid_search2 = GridSearchCV(extra_trees_clf, param_grid, cv =5,
                          scoring='neg_mean_squared_error',
                          return_train_score=True)
grid_search2.fit(X_train, y_train)
grid_search2.best_params_

{'max_depth': 12, 'n_estimators': 450}

In [24]:
# mnist.1에서 gridsearch를 통해 찾은 최적값 대입
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

rnd_clf = RandomForestClassifier(n_estimators =450, max_depth = 12, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators =450, max_depth = 12, random_state=42)
knn_clf = KNeighborsClassifier(n_jobs=-1, weights='distance', n_neighbors=4)
svc_clf =  Pipeline([("scaler", StandardScaler()), 
                    ("svc_clf", SVC(kernel="poly", degree=3, gamma='auto', C=1))
                   ])

voting_clf = VotingClassifier(
    estimators=[('etc', extra_trees_clf), ('rf', rnd_clf),  
                ('svc', svc_clf), ('knn', knn_clf)],voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('etc',
                              ExtraTreesClassifier(bootstrap=False,
                                                   ccp_alpha=0.0,
                                                   class_weight=None,
                                                   criterion='gini',
                                                   max_depth=12,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                  

In [20]:
# mnist.1에서 gridsearch를 통해 찾은 최적값 대입
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_leaf_nodes = 40), n_estimators=200,
    algorithm="SAMME.R", learning_rate=1, random_state=42)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=40,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                         

In [28]:
from sklearn.metrics import accuracy_score

for clf in (extra_trees_clf, rnd_clf, svc_clf, knn_clf, voting_clf, ada_clf):
    clf.fit(X_train, y_train)
    y_pred=clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test, y_pred))

ExtraTreesClassifier 0.9323
RandomForestClassifier 0.93765
Pipeline 0.96645
KNeighborsClassifier 0.97415
VotingClassifier 0.95985
AdaBoostClassifier 0.88905


In [29]:
#KNeighborsClassifier 이용한 CONFUSION MATRIX
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_train_pred_knn = cross_val_predict(knn_clf, X_train, y_train, cv=5)

print(confusion_matrix(y_train, y_train_pred_knn))

[[4894    3    5    0    0    4   16    2    5    3]
 [   1 5646   10    0    3    0    2   11    2    3]
 [  31   33 4779   16    8    3    9   66   17    6]
 [   8   12   36 4900    2   50    5   32   36   20]
 [   5   30    1    1 4696    0   18    8    2   98]
 [   9    6    1   45    3 4360   49    5    7   21]
 [  16   10    1    0    7   20 4894    0    3    0]
 [   3   41   14    1   12    3    0 5053    3   45]
 [  13   52   13   53   21   53   24   12 4548   53]
 [  10    4    5   20   33   10    4   54    7 4841]]


In [ ]:
#KNeighborsClassifier 이용한 ROC 곡선
from sklearn.metrics import roc_curve
y_train_5 = (y_train == 5)
y_probas_kneighbor = cross_val_predict(knn_clf, X_train, y_train_5, cv=3,
                             method="predict_proba")
y_scores_kneighbor = y_probas_kneighbor[:, 1]
fpr, tpr, threshold = roc_curve(y_train_5, y_scores_kneighbor)

def plot_roc_curve(fpr, tpr, label=None):
    plt.title('ROC curve')
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0,1],[0,1],"k--")
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')

plot_roc_curve(fpr, tpr)
plt.show()

In [ ]:
#ROC 곡선 비교
#RandomForestClassifier
y_probas_forest = cross_val_predict(rnd_clf, X_train, y_train_5, cv=3,
                                    method="predict_proba")
y_scores_forest = y_probas_forest[:, 1] 
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5,y_scores_forest)

#ExtraTreesClassifier
y_probas_extra = cross_val_predict(extra_trees_clf, X_train, y_train_5, cv=3,
                                    method="predict_proba")
y_scores_extra = y_probas_extra[:, 1] 
fpr_extra, tpr_extra, thresholds_extra = roc_curve(y_train_5,y_scores_extra)

plot_roc_curve(fpr, tpr)
plt.plot(fpr, tpr,"g", label="knn")
plot_roc_curve(fpr_forest, tpr_forest)
plt.plot(fpr_forest, tpr_forest,"b", label="rnd")
plot_roc_curve(fpr_extra, tpr_extra)
plt.plot(fpr_extra, tpr_extra,"r", label="extra_trees")
plt.legend(loc="lower right")
plt.show()